# Predicting Customer Churn in Telecommunication Operators
Customer turnover refers to a decision made by the customer on the term of business relationship. Customer loyalty and customer turnover always add up to 100%. If a company has a 60% loyalty rate, then customer loss taxes are 40%. According to the 80/20 customer profitability rule, 20% of customers are generating 80% of revenue. Therefore, it is very important to predict the users who are likely to abandon the business relationship and the factors that affect how the customer's decisions.<br><br>
In this project, I predicted Customer Churn at a Telecommunications Operator using pyspark and frameworks (Pandas, Numpy, scipy and Scikit-Learn).

In [78]:
# Importing libraries and frameworks
from pyspark.sql.functions import col,sum
from scipy.stats import skew, kurtosis

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

## Importing dataset

In [2]:
df_train = spark.read.csv("data/projeto4_telecom_treino.csv", header=True)
df_test = spark.read.csv("data/projeto4_telecom_teste.csv", header=True)

## Exploratory Analysis

In [10]:
# Checking train data
display(df_train.head(5))

(3333, 21)


[Row(_c0='1', state='KS', account_length='128', area_code='area_code_415', international_plan='no', voice_mail_plan='yes', number_vmail_messages='25', total_day_minutes='265.1', total_day_calls='110', total_day_charge='45.07', total_eve_minutes='197.4', total_eve_calls='99', total_eve_charge='16.78', total_night_minutes='244.7', total_night_calls='91', total_night_charge='11.01', total_intl_minutes='10', total_intl_calls='3', total_intl_charge='2.7', number_customer_service_calls='1', churn='no'),
 Row(_c0='2', state='OH', account_length='107', area_code='area_code_415', international_plan='no', voice_mail_plan='yes', number_vmail_messages='26', total_day_minutes='161.6', total_day_calls='123', total_day_charge='27.47', total_eve_minutes='195.5', total_eve_calls='103', total_eve_charge='16.62', total_night_minutes='254.4', total_night_calls='103', total_night_charge='11.45', total_intl_minutes='13.7', total_intl_calls='3', total_intl_charge='3.7', number_customer_service_calls='1', chu

In [11]:
# Ckecking size of datasets
print((df_train.count(), len(df_train.columns)))
print((df_test.count(), len(df_train.columns)))

(3333, 21)
(1667, 21)


In [24]:
# Checking for missing values on train and test datasets
df_train.select(*(sum(col(c).isNull().cast("int")).alias(c) for c in df_train.columns)).show()
df_test.select(*(sum(col(c).isNull().cast("int")).alias(c) for c in df_train.columns)).show()

+---+-----+--------------+---------+------------------+---------------+---------------------+-----------------+---------------+----------------+-----------------+---------------+----------------+-------------------+-----------------+------------------+------------------+----------------+-----------------+-----------------------------+-----+
|_c0|state|account_length|area_code|international_plan|voice_mail_plan|number_vmail_messages|total_day_minutes|total_day_calls|total_day_charge|total_eve_minutes|total_eve_calls|total_eve_charge|total_night_minutes|total_night_calls|total_night_charge|total_intl_minutes|total_intl_calls|total_intl_charge|number_customer_service_calls|churn|
+---+-----+--------------+---------+------------------+---------------+---------------------+-----------------+---------------+----------------+-----------------+---------------+----------------+-------------------+-----------------+------------------+------------------+----------------+-----------------+--------

In [25]:
df_train.columns

['_c0',
 'state',
 'account_length',
 'area_code',
 'international_plan',
 'voice_mail_plan',
 'number_vmail_messages',
 'total_day_minutes',
 'total_day_calls',
 'total_day_charge',
 'total_eve_minutes',
 'total_eve_calls',
 'total_eve_charge',
 'total_night_minutes',
 'total_night_calls',
 'total_night_charge',
 'total_intl_minutes',
 'total_intl_calls',
 'total_intl_charge',
 'number_customer_service_calls',
 'churn']

In [3]:
# Compute numerical data summary statistics
df_train_num = df_train.select(['account_length', 'number_vmail_messages', 'total_day_minutes',
                        'total_day_calls', 'total_day_charge', 'total_eve_minutes', 
                        'total_eve_calls', 'total_eve_charge', 'total_night_minutes', 
                        'total_night_calls', 'total_night_charge', 'total_intl_minutes', 
                        'total_intl_calls', 'total_intl_charge', 'number_customer_service_calls'])

summary_df = df_train_num.describe().toPandas()

s = ['account_length', 'number_vmail_messages', 'total_day_minutes',
                        'total_day_calls', 'total_day_charge', 'total_eve_minutes', 
                        'total_eve_calls', 'total_eve_charge', 'total_night_minutes', 
                        'total_night_calls', 'total_night_charge', 'total_intl_minutes', 
                        'total_intl_calls', 'total_intl_charge', 'number_customer_service_calls']


df = pd.DataFrame([pd.to_numeric(summary_df[col]) for col in s]).T.round(2)
idx = 0
df.insert(loc=idx, column='summary', value=summary_df.summary.values)

display(df)

,summary,account_length,number_vmail_messages,total_day_minutes,total_day_calls,total_day_charge,total_eve_minutes,total_eve_calls,total_eve_charge,total_night_minutes,total_night_calls,total_night_charge,total_intl_minutes,total_intl_calls,total_intl_charge,number_customer_service_calls
0,count,3333.00,3333.00,3333.00,3333.00,3333.00,3333.00,3333.00,3333.00,3333.00,3333.00,3333.00,3333.00,3333.00,3333.00,3333.00
1,mean,101.06,8.10,179.78,100.44,30.56,200.98,100.11,17.08,200.87,100.11,9.04,10.24,4.48,2.76,1.56
2,stddev,39.82,13.69,54.47,20.07,9.26,50.71,19.92,4.31,50.57,19.57,2.28,2.79,2.46,0.75,1.32
3,min,1.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,100.30,100.00,1.04,0.00,0.00,0.00,0.00
4,max,99.00,9.00,99.90,99.00,9.93,99.50,99.00,9.95,99.30,99.00,9.99,9.90,9.00,5.40,9.00


In [54]:
# converting columns to numeric and calculating median
for col_name in df_train_num.columns:
    df_train_num = df_train_num.withColumn(col_name, df_train_num[col_name].cast('float'))
    
median = [df_train_num.approxQuantile(col, [0.5], 0.0)[0] for col in df_train_num.columns]
first_quartile = [df_train_num.approxQuantile(col, [0.25], 0.0)[0] for col in df_train_num.columns]
third_quartile = [df_train_num.approxQuantile(col, [0.75], 0.0)[0] for col in df_train_num.columns]
#med = pd.Series({'median': median})

In [79]:
# Getting mean absolute deviation
def mad(col, axis=None):
    data = [int(row[col]) for row in df_train_num.select(col).collect()]
    return np.mean(np.absolute(data - np.mean(data, axis)), axis)

mad = [mad(col) for col in df_train_num.columns]

# Getting skewness
skew_list = [skew([int(row[col]) for row in df_train_num.select(col).collect()])
            for col in df_train_num.columns]

# Getting kurtosis
kurt = [kurtosis([int(row[col]) for row in df_train_num.select(col).collect()])
            for col in df_train_num.columns]